In [2]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import itertools

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [18]:
import time

## 导入数据
-  label 文件一共有 79572 行
- 各种维度混合在一起
- 我们的目的是切分开各种维度, 进行训练和模拟
- 那么首先取出个标签的图片进行聚类

In [3]:
df_train = pd.read_csv('data/train2/Annotations/label.csv', header=None)
df_train.columns = ['image_id', 'class', 'label']
df_train.head()

,image_id,class,label
0,Images/collar_design_labels/b23a3fe060d71bed77...,collar_design_labels,ynnnn
1,Images/collar_design_labels/0ef580b4deabcd9fa4...,collar_design_labels,ynnnn
2,Images/collar_design_labels/87ccc33937821a97ad...,collar_design_labels,nnynn
3,Images/collar_design_labels/26caac7d1f1b36fb9d...,collar_design_labels,ynnnn
4,Images/collar_design_labels/396ab4e7cbc6798100...,collar_design_labels,nynnn


In [4]:
classes = ['collar_design_labels', 'neckline_design_labels', 'skirt_length_labels',
          'sleeve_length_labels', 'neck_design_labels', 'coat_length_labels', 'lapel_design_labels',
          'pant_length_labels']

In [5]:
cur_class = classes[5]
df_load = df_train[(df_train['class'] == cur_class)].copy()
df_load.reset_index(inplace=True)
del df_load['index']

print('{0}: {1}'.format(cur_class, len(df_load)))
df_load.head()

coat_length_labels: 14454


,image_id,class,label
0,Images/coat_length_labels/55eb58f477d33272aff3...,coat_length_labels,nnynnnnn
1,Images/coat_length_labels/6324fc9b689445b9dbc5...,coat_length_labels,nnynnnnn
2,Images/coat_length_labels/c82f855a3115f6ec5520...,coat_length_labels,nnnnnynn
3,Images/coat_length_labels/a9b54e901758ee46c63a...,coat_length_labels,nnnnynnn
4,Images/coat_length_labels/bf4ac23424dbed2abc17...,coat_length_labels,nnnnnnny


In [6]:
n = len(df_load)
n_class = len(df_load['label'][0])
width = 299 # 定义图片大小

X = np.zeros((n, width, width, 3), dtype=np.uint8)
y = np.zeros((n, n_class), dtype=np.uint8)

In [7]:
for i in tqdm(range(n)):
    tmp_label = df_load['label'][i]
    if len(tmp_label) > n_class:
        print(df_load['image_id'][i])
    X[i] = cv2.resize(cv2.imread('data/train2/{0}'.format(df_load['image_id'][i])), (width, width))
    y[i][tmp_label.find('y')] = 1

100%|██████████| 14454/14454 [01:30<00:00, 159.38it/s]


In [8]:
X.shape, y.shape

((14454, 299, 299, 3), (14454, 8))

## 提取特征

In [8]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import *
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import *
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import RMSprop
import keras

Using TensorFlow backend.


In [9]:
prefix_cls = cur_class.split('_')[0]

In [10]:
save_path='{0}_Xception_try_weights.h5'.format(prefix_cls)
load_path='{0}_Xception_try_weights.h5'.format(prefix_cls)

In [11]:
cnn_model = Xception(include_top=False, input_shape=(width, width, 3), weights='imagenet')

In [12]:
inputs = Input((width, width, 3))

x = inputs
x = Lambda(preprocess_input, name='preprocessing')(x)
x = cnn_model(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(n_class, activation='softmax', name='softmax')(x)

model = Model(inputs, x)

In [12]:
model.load_weights(load_path)

In [13]:
#冻结block14层之前
# set_trainable = False
# for layer in model.layers[2].layers:#model.layers[2]是xception
#     if layer.name == 'block14_sepconv1':
#         set_trainable = True
#     if set_trainable:
#         layer.trainable = True
#     else:
#         layer.trainable = False

## 划分训练/测试集

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)
X_train.shape, y_train.shape, X_valid.shape

((13008, 299, 299, 3), (13008, 8), (1446, 299, 299, 3))

In [14]:
epochs = 32
batch_size = 32

In [15]:

n_batch=X_train.shape[0]//batch_size # 之后next迭代的次数，batch_size是迭代一次的选取的图片

X_train=X_train[:n_batch*batch_size] #取可以被批次整除
X_train=X_train.reshape((n_batch,batch_size,width,width,3))# shape(n_batch,batch_size,width,width,3),5维
y_train=y_train[:n_batch*batch_size]
y_train=y_train.reshape((n_batch,batch_size,-1))# shape（n_batch,batch_size,-1）3维度

In [16]:
# Compile the model
adam = Adam(lr=0.001) 

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
# Set a learning rate annealer
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
#                                             patience=3,
#                                             verbose=1,
#                                             factor=0.1,
#                                             min_lr=0.00001)


数据增强
--------------
对训练数据增强


In [17]:
datagen = ImageDataGenerator(
        featurewise_center =False ,# set input mean to 0 over the dataset
        samplewise_center = False ,# set each sample mean to 0
        featurewise_std_normalization = False, # divide inputs by std of the dataset
        samplewise_std_normalization = False, # divide each input by its std
        zca_whitening = False, # apply ZCA whitening
        rotation_range = 10, # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # randomly zoom image
        width_shift_range = 0.2, # randomly shift images horizontally (fraction of total width)
        height_shift_range = 0.2, # randomly shift images vertivally (fraction of total heigth)
        horizontal_flip = True, # randomly flip images
        vertical_flip = False,
        shear_range = 0.1,
        fill_mode = 'constant',
        cval = 0)

#datagen.fit(X_train)


In [ ]:
val_acc_last=0
i=0

for epoch in range(epochs):
    time_begin=time.time()
    loss_sum=0
    acc_sum=0
    iter_train=iter(zip(X_train,y_train)) #将x_train和y_train的合并
    print('epoch:',epoch)
    for step in range(n_batch):
    
        X_batch,y_batch=next(iter_train) #每次一个批次的
        datagen.fit(X_batch,seed=123)
        for X_batch, y_batch in datagen.flow(X_batch, y_batch,batch_size=batch_size):#这里只取图片预处理循环一次，否则会一直迭代下去
            break
        history=model.fit(X_batch, y_batch,epochs=1,verbose=0)
        if step % 25==0:
            print('epoch:',epoch,'------','All steps is:',n_batch,'-----','this is step:',step)
            print('train LOSS is :',history.history['loss'][0],'train ACC is :',history.history['acc'][0])
        loss_sum +=history.history['loss'][0]
        acc_sum  +=history.history['acc'][0]
    print('This epoch:',epoch,'---','the average LOSS is:',loss_sum/n_batch,'---','the average ACC is:', acc_sum/n_batch)
    val_acc=model.evaluate(X_valid,y_valid)#评估
    print('val_loss is :',val_acc[0],'---','val_acc is :',val_acc[1])
    if val_acc[1] > val_acc_last:
        i +=1
        print('model save {} times'.format(i))
        model.save_weights('{0}_Xception_try_weights.h5'.format(prefix_cls))
        val_acc_last=val_acc[1]
    time_end=time.time()
    print('This epoch take time is :',time_end-time_begin)

epoch: 0
epoch: 0 ------ All steps is: 406 ----- this is step: 0
train LOSS is : 1.4181246757507324 train ACC is : 0.28125
epoch: 0 ------ All steps is: 406 ----- this is step: 100
train LOSS is : 1.2926833629608154 train ACC is : 0.40625
epoch: 0 ------ All steps is: 406 ----- this is step: 200
train LOSS is : 1.1605550050735474 train ACC is : 0.4375
epoch: 0 ------ All steps is: 406 ----- this is step: 300
train LOSS is : 1.1597620248794556 train ACC is : 0.71875


## 加载模型

In [20]:
df_test = pd.read_csv('data/week-rank/Tests/question.csv', header=None)
df_test.columns = ['image_id', 'class', 'x']
del df_test['x']
df_test.head()

,image_id,class
0,Images/collar_design_labels/1078bc9a4fa91aef69...,collar_design_labels
1,Images/collar_design_labels/272c5d069581efc94f...,collar_design_labels
2,Images/collar_design_labels/2e67922d82216cc756...,collar_design_labels
3,Images/collar_design_labels/d9726ceee17c7ee811...,collar_design_labels
4,Images/collar_design_labels/1b7de7f9def99a5aa6...,collar_design_labels


In [21]:
df_load = df_test[(df_test['class'] == cur_class)].copy()
df_load.reset_index(inplace=True)
del df_load['index']

print('{0}: {1}'.format(cur_class, len(df_load)))
df_load.head()

neck_design_labels: 1368


,image_id,class
0,Images/neck_design_labels/124df715c45a3f76b6e0...,neck_design_labels
1,Images/neck_design_labels/3eef78e3ecf37deb4f9f...,neck_design_labels
2,Images/neck_design_labels/e80b32f77ab355afe1ca...,neck_design_labels
3,Images/neck_design_labels/e33c717ea13b7ac83305...,neck_design_labels
4,Images/neck_design_labels/9a4313e3f13cbc121e5c...,neck_design_labels


In [22]:
n = len(df_load)
X_test = np.zeros((n, width, width, 3), dtype=np.uint8)

for i in tqdm(range(n)):
    X_test[i] = cv2.resize(cv2.imread('data/week-rank/{0}'.format(df_load['image_id'][i])), (width, width))


100%|██████████| 1368/1368 [00:05<00:00, 258.76it/s]


In [24]:
model.load_weights(save_path)

In [25]:
test_np = model.predict(X_test, batch_size=256)

In [26]:
test_np.shape

(1368, 5)

In [27]:
result = []

for i, row in df_load.iterrows():
    tmp_list = test_np[i]
    tmp_result = ''
    for tmp_ret in tmp_list:
        tmp_result += '{:.4f};'.format(tmp_ret)
        
    result.append(tmp_result[:-1])

df_load['result'] = result
df_load.head()

,image_id,class,result
0,Images/neck_design_labels/124df715c45a3f76b6e0...,neck_design_labels,0.0129;0.4206;0.0348;0.0507;0.4811
1,Images/neck_design_labels/3eef78e3ecf37deb4f9f...,neck_design_labels,0.0931;0.0663;0.0033;0.0060;0.8313
2,Images/neck_design_labels/e80b32f77ab355afe1ca...,neck_design_labels,0.2992;0.0048;0.3887;0.3049;0.0024
3,Images/neck_design_labels/e33c717ea13b7ac83305...,neck_design_labels,0.0042;0.0918;0.0089;0.0128;0.8823
4,Images/neck_design_labels/9a4313e3f13cbc121e5c...,neck_design_labels,0.0049;0.0002;0.7056;0.2892;0.0001


In [30]:
df_load.to_csv('result/{}__0502.csv'.format(prefix_cls), header=None, index=False)